In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter
import time

from model.nets import Siren
from model.flow import PipeFlow
from model.integrator import Integrator

In [2]:
Wo = 5
Pn = [  complex(-0.5060,0.1245),
        complex(0.0802,0.0886),
        complex(0.0591,0.0251),
        complex(0.0228,0.0244),
        complex(0.0141,0.0071),
        complex(0.0112,0.0107),
        complex(0.0065,0.0056),
        complex(0.0067,0.0050),
        complex(0.0030,0.0040),
        complex(0.0044,0.0030),
        complex(0.0000,0.3710),
        complex(0.1493,0.0754),
        complex(0.0161,0.0103),
        complex(0.0166,0.0082),
        complex(-0.0002,0.0046),
        complex(0.0063,0.0001),
        complex(0.0008,0.0013),
        complex(0.0021,-0.0012),
        complex(0.0009,0.0015),
        complex(0.0001,-0.0001)]
pipe = PipeFlow(Po=-0.05,Pn=Pn,Wo=Wo)

In [3]:
# xb = np.linspace(0,8,4*Nx)
# yb = np.ones_like(xb)
# xb = np.concatenate((xb,xb),axis=0)
# yb = np.concatenate((yb,yb*-1),axis=0)
# tb = np.zeros_like(xb)
# ub = np.zeros_like(xb)
# vb = np.zeros_like(xb)
# dpdyb = np.zeros_like(xb)

# Xb = np.concatenate((xb.reshape((8*Nx,1)),yb.reshape((8*Nx,1)),tb.reshape((8*Nx,1))),axis=1)
# Ub = np.concatenate((ub.reshape((8*Nx,1)),vb.reshape((8*Nx,1))),axis=1)
# Xb = torch.from_numpy(Xb).float().cuda()
# Ub = torch.from_numpy(Ub).float().cuda()
# dpdyb = torch.from_numpy(dpdyb).float().cuda()

In [4]:
net = Siren(in_features=4, out_features=4, hidden_features=512, 
                  hidden_layers=5, outermost_linear=True)
net.cuda()

Siren(
  (net): Sequential(
    (0): SineLayer(
      (linear): Linear(in_features=4, out_features=512, bias=True)
    )
    (1): SineLayer(
      (linear): Linear(in_features=512, out_features=512, bias=True)
    )
    (2): SineLayer(
      (linear): Linear(in_features=512, out_features=512, bias=True)
    )
    (3): SineLayer(
      (linear): Linear(in_features=512, out_features=512, bias=True)
    )
    (4): SineLayer(
      (linear): Linear(in_features=512, out_features=512, bias=True)
    )
    (5): SineLayer(
      (linear): Linear(in_features=512, out_features=512, bias=True)
    )
    (6): Linear(in_features=512, out_features=4, bias=True)
  )
)

In [11]:
total_steps = 5000

optim = torch.optim.Adam(lr=1e-4, params=net.parameters())

L = np.array(((-1,1),(-1,1),(0,8),(0,1)))
ig = Integrator(Nd=4,L=L,Np=2**12)

lam_div = 1
lam_mm  = 1

writer = SummaryWriter('runs/pipe_flowsup_ns1d-'+str(np.round(time.time())))

for step in range(total_steps):
    
    
#     func = lambda X: ((net(X)-pipe.fields(X))**2).sum(axis=1)

#     dudxp = net.jacobian(X,0,0)
#     dvdyp = net.jacobian(X,1,1)
#     dwdzp = net.jacobian(X,2,2)
    
#     mseloss = ((F - Fp)**2).mean()
#     divloss = ((dudxp+dvdyp+dwdzp)**2).mean()
#     loss    = mseloss+lam*divloss
    
#     loss = ig.integrate(func,reltol=5e-3,maxiter=1e3)
    
    # Sample points
    X = ig.points()
    
    # Fields
    F = pipe.fields(X)
    u = F[:,0]
    v = F[:,2]
    w = F[:,2]
    
    # First derivatives
    dudx = net.jacobian(X,0,0)
    dudy = net.jacobian(X,0,1)
    dudz = net.jacobian(X,0,2)
    dudt = net.jacobian(X,0,3)
    
    dvdx = net.jacobian(X,1,0)
    dvdy = net.jacobian(X,1,1)
    dvdz = net.jacobian(X,1,2)
    dvdt = net.jacobian(X,1,3)
    
    dwdx = net.jacobian(X,2,0)
    dwdy = net.jacobian(X,2,1)
    dwdz = net.jacobian(X,2,2)
    dwdt = net.jacobian(X,2,3)
    
    dpdx = net.jacobian(X,3,0)
    dpdy = net.jacobian(X,3,1)
    dpdz = net.jacobian(X,3,2)
    
    # Second derivatives
    d2udx2 = net.hessian(X,0,0,0)
    d2udy2 = net.hessian(X,0,1,1)
    d2udz2 = net.hessian(X,0,2,2)
    
    d2vdx2 = net.hessian(X,1,0,0)
    d2vdy2 = net.hessian(X,1,1,1)
    d2vdz2 = net.hessian(X,1,2,2)
    
    d2wdx2 = net.hessian(X,2,0,0)
    d2wdy2 = net.hessian(X,2,1,1)
    d2wdz2 = net.hessian(X,2,2,2)
    
    # Momentum
    mmx = dudx + u*dudx + v*dudy + w*dudz + dpdx - 1/Wo**2*(d2udx2 + d2udy2 + d2udz2)
    mmy = dvdx + u*dvdx + v*dvdy + w*dvdz + dpdy - 1/Wo**2*(d2vdx2 + d2vdy2 + d2vdz2)
    mmz = dwdx + u*dwdx + v*dwdy + w*dwdz + dpdz - 1/Wo**2*(d2wdx2 + d2wdy2 + d2wdz2)
    
    # Continuity
    div = dudx + dvdy + dwdz
    
    # Loss Terms
    mseloss = ((net(X)-F)**2).mean()
    divloss = (div**2).mean()
    mmloss  = (mmx**2+mmy**2+mmz**2).mean()
    
#     loss = mseloss + lam_div*divloss + lam_mm*mmloss
    loss = mseloss + lam_div*divloss
    
    writer.add_scalar('MSE Loss',
                        mseloss.item(),
                        step)
    writer.add_scalar('Divergence Loss',
                        divloss.item(),
                        step)
    writer.add_scalar('Momentum Loss',
                        mmloss.item(),
                        step)
    writer.add_scalar('Total Loss',
                        loss.item(),
                        step)

    optim.zero_grad()
    loss.backward()
    optim.step()
    
    print("Loss " + str(step) + ": " + str(loss.item()))

Loss 0: 0.028991686180233955
Loss 1: 0.10047820210456848
Loss 2: 0.05597878247499466
Loss 3: 0.06671324372291565
Loss 4: 0.06260371208190918
Loss 5: 0.05769031494855881
Loss 6: 0.05502700060606003
Loss 7: 0.05934951826930046
Loss 8: 0.05073578655719757
Loss 9: 0.047261592000722885
Loss 10: 0.05157905071973801
Loss 11: 0.05377773940563202
Loss 12: 0.04833653196692467
Loss 13: 0.046037230640649796
Loss 14: 0.04488750919699669
Loss 15: 0.04767150431871414
Loss 16: 0.04590035602450371
Loss 17: 0.044654905796051025
Loss 18: 0.04189611226320267
Loss 19: 0.041007764637470245
Loss 20: 0.041733331978321075
Loss 21: 0.040548354387283325
Loss 22: 0.040779951959848404
Loss 23: 0.0401897206902504
Loss 24: 0.039392489939928055
Loss 25: 0.03825885429978371
Loss 26: 0.03998102992773056
Loss 27: 0.03689468652009964
Loss 28: 0.03674223646521568
Loss 29: 0.03960568830370903
Loss 30: 0.037268538028001785
Loss 31: 0.03780417889356613
Loss 32: 0.03693182393908501
Loss 33: 0.03720531612634659
Loss 34: 0.0365

Loss 275: 0.0314553938806057
Loss 276: 0.03148644044995308
Loss 277: 0.032026126980781555
Loss 278: 0.032578594982624054
Loss 279: 0.03203355893492699
Loss 280: 0.03269418329000473
Loss 281: 0.03523539379239082
Loss 282: 0.03651675581932068
Loss 283: 0.03524798899888992
Loss 284: 0.04048764333128929
Loss 285: 0.04049910232424736
Loss 286: 0.03888662904500961
Loss 287: 0.03991616517305374
Loss 288: 0.044012635946273804
Loss 289: 0.04065961763262749
Loss 290: 0.04178326576948166
Loss 291: 0.048296328634023666
Loss 292: 0.04881542548537254
Loss 293: 0.047010134905576706
Loss 294: 0.05053158476948738
Loss 295: 0.051027633249759674
Loss 296: 0.05209317058324814
Loss 297: 0.04909228906035423
Loss 298: 0.05111994594335556
Loss 299: 0.05677446722984314
Loss 300: 0.04973769187927246
Loss 301: 0.054310400038957596
Loss 302: 0.05336908623576164
Loss 303: 0.054718244820833206
Loss 304: 0.0523172989487648
Loss 305: 0.05287706106901169
Loss 306: 0.05122405290603638
Loss 307: 0.0517776682972908
Loss 

Loss 546: 0.04332321882247925
Loss 547: 0.044246938079595566
Loss 548: 0.04180968180298805
Loss 549: 0.043997954577207565
Loss 550: 0.041633784770965576
Loss 551: 0.04316529631614685
Loss 552: 0.041501887142658234
Loss 553: 0.04056589677929878
Loss 554: 0.043218888342380524
Loss 555: 0.04166660085320473
Loss 556: 0.04307598993182182
Loss 557: 0.04171241074800491
Loss 558: 0.041200608015060425
Loss 559: 0.039146389812231064
Loss 560: 0.03934953361749649
Loss 561: 0.04085272550582886
Loss 562: 0.03930840641260147
Loss 563: 0.04064682498574257
Loss 564: 0.03906990587711334
Loss 565: 0.03942374512553215
Loss 566: 0.03719382360577583
Loss 567: 0.03861053287982941
Loss 568: 0.04087095335125923
Loss 569: 0.03763872757554054
Loss 570: 0.03837102651596069
Loss 571: 0.03675369545817375
Loss 572: 0.03629233315587044
Loss 573: 0.035867199301719666
Loss 574: 0.037433795630931854
Loss 575: 0.03765552490949631
Loss 576: 0.03489591181278229
Loss 577: 0.03748803585767746
Loss 578: 0.03684016317129135
L

Loss 817: 0.03295695409178734
Loss 818: 0.0324990376830101
Loss 819: 0.03266531229019165
Loss 820: 0.03329260274767876
Loss 821: 0.032293353229761124
Loss 822: 0.03148063272237778
Loss 823: 0.033144522458314896
Loss 824: 0.03273024037480354
Loss 825: 0.031269680708646774
Loss 826: 0.0313858687877655
Loss 827: 0.032464705407619476
Loss 828: 0.030563142150640488
Loss 829: 0.030923936516046524
Loss 830: 0.03064611554145813
Loss 831: 0.03156512603163719
Loss 832: 0.0313887894153595
Loss 833: 0.030849458649754524
Loss 834: 0.030762003734707832
Loss 835: 0.030909111723303795
Loss 836: 0.030257156118750572
Loss 837: 0.03179437294602394
Loss 838: 0.03112446330487728
Loss 839: 0.029791807755827904
Loss 840: 0.029405182227492332
Loss 841: 0.03040340729057789
Loss 842: 0.0284589696675539
Loss 843: 0.029551127925515175
Loss 844: 0.030891621485352516
Loss 845: 0.03018355369567871
Loss 846: 0.03144265338778496
Loss 847: 0.030188430100679398
Loss 848: 0.027691874653100967
Loss 849: 0.0306231305003166

Loss 1085: 0.03654731437563896
Loss 1086: 0.03861230984330177
Loss 1087: 0.036188237369060516
Loss 1088: 0.03716781735420227
Loss 1089: 0.0358431302011013
Loss 1090: 0.036155492067337036
Loss 1091: 0.03693477436900139
Loss 1092: 0.033599164336919785
Loss 1093: 0.03648494556546211
Loss 1094: 0.03545602783560753
Loss 1095: 0.03487064689397812
Loss 1096: 0.03483317047357559
Loss 1097: 0.03315195441246033
Loss 1098: 0.035436708480119705
Loss 1099: 0.03355559706687927
Loss 1100: 0.03520527109503746
Loss 1101: 0.034322626888751984
Loss 1102: 0.033604543656110764
Loss 1103: 0.033921848982572556
Loss 1104: 0.03404891863465309
Loss 1105: 0.033504024147987366
Loss 1106: 0.0326552540063858
Loss 1107: 0.0328650176525116
Loss 1108: 0.03260482847690582
Loss 1109: 0.03274046629667282
Loss 1110: 0.03338818997144699
Loss 1111: 0.03251403570175171
Loss 1112: 0.03395140543580055
Loss 1113: 0.033357806503772736
Loss 1114: 0.03203481435775757
Loss 1115: 0.031645528972148895
Loss 1116: 0.03162940591573715
L

KeyboardInterrupt: 

In [ ]:
Nxt = 32
Nzt = 4*Nxt
Ntt = 64

xt,yt,zt,tt = np.meshgrid(np.linspace(-1,1,Nxt),np.linspace(0,0,1),np.linspace(0,4,Nzt),np.linspace(0,1,Ntt))
ut,vt,wt = pipe.velocity(xt,yt,zt,tt)
pt = pipe.pressure(xt,yt,zt,tt)
ut = np.squeeze(ut[0,:,:,:])
vt = np.squeeze(vt[0,:,:,:])
wt = np.squeeze(wt[0,:,:,:])
pt = np.squeeze(pt[0,:,:,:])


Xt = np.concatenate((xt.reshape((Nxt*Nzt*Ntt,1)),yt.reshape((Nxt*Nzt*Ntt,1)),zt.reshape((Nxt*Nzt*Ntt,1)),tt.reshape((Nxt*Nzt*Ntt,1))),axis=1)
Xt = torch.from_numpy(Xt).float().cuda()
Fpt = net(Xt)

Fpt = Fpt.cpu().detach().numpy().reshape((Nxt,1,Nzt,Ntt,4))
upt = np.squeeze(Fpt[:,0,:,:,0])
vpt = np.squeeze(Fpt[:,0,:,:,1])
wpt = np.squeeze(Fpt[:,0,:,:,2])
ppt = np.squeeze(Fpt[:,0,:,:,3])
for tn in np.arange(Ntt):
    fig, axes = plt.subplots(4,3, figsize=(16.5,5.5))
    axes[0,0].imshow(ut[:,:,tn],vmin=-1,vmax=1)
    axes[0,1].imshow(upt[:,:,tn],vmin=-1,vmax=1)
    axes[0,2].imshow(upt[:,:,tn]-ut[:,:,tn],vmin=-0.1,vmax=0.1)
    
    axes[1,0].imshow(vt[:,:,tn],vmin=-1,vmax=1)
    axes[1,1].imshow(vpt[:,:,tn],vmin=-1,vmax=1)
    axes[1,2].imshow(vpt[:,:,tn]-vt[:,:,tn],vmin=-0.1,vmax=0.1)
    
    axes[2,0].imshow(wt[:,:,tn],vmin=-1,vmax=1)
    axes[2,1].imshow(wpt[:,:,tn],vmin=-1,vmax=1)
    axes[2,2].imshow(wpt[:,:,tn]-wt[:,:,tn],vmin=-0.1,vmax=0.1)
    
    axes[3,0].imshow(pt[:,:,tn],vmin=-1,vmax=1)
    axes[3,1].imshow(ppt[:,:,tn],vmin=-1,vmax=1)
    axes[3,2].imshow(ppt[:,:,tn]-pt[:,:,tn],vmin=-0.1,vmax=0.1)
    
    
    plt.show()

In [14]:
2**20

1048576